# Quick Start in 30s

**Home GitHub Repository:** [LEANN on GitHub](https://github.com/yichuan-w/LEANN)

**Important for Colab users:** Set your runtime type to T4 GPU for optimal performance. Go to Runtime → Change runtime type → Hardware accelerator → T4 GPU.

In [ ]:
# install this if you are using colab
! uv pip install leann-core leann-backend-hnsw --no-deps
! uv pip install leann --no-deps
# For Colab environment, we need to set some environment variables
import os

os.environ["LEANN_LOG_LEVEL"] = "INFO"  # Enable more detailed logging

In [ ]:
from pathlib import Path

INDEX_DIR = Path("./").resolve()
INDEX_PATH = str(INDEX_DIR / "demo.leann")

## Build the index

In [ ]:
from leann.api import LeannBuilder

builder = LeannBuilder(backend_name="hnsw")
builder.add_text("C# is a powerful programming language and it is good at game development")
builder.add_text(
    "Python is a powerful programming language and it is good at machine learning tasks"
)
builder.add_text("Machine learning transforms industries")
builder.add_text("Neural networks process complex data")
builder.add_text("Leann is a great storage saving engine for RAG on your MacBook")
builder.build_index(INDEX_PATH)

Batches: 100%|██████████| 1/1 [00:00<00:00, 36.43it/s]
INFO:leann_backend_hnsw.hnsw_backend:INFO: Converting HNSW index to CSR-pruned format...


M: 64 for level: 0
Starting conversion: index.index -> index.csr.tmp
[0.00s] Reading Index HNSW header...
[0.00s]   Header read: d=768, ntotal=5
[0.00s] Reading HNSW struct vectors...
  Reading vector (dtype=<class 'numpy.float64'>, fmt='d')... Count=6, Bytes=48
[0.00s]   Read assign_probas (6)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=7, Bytes=28
[0.14s]   Read cum_nneighbor_per_level (7)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=5, Bytes=20
[0.24s]   Read levels (5)
[0.33s]   Probing for compact storage flag...
[0.33s]   Found compact flag: False
[0.33s]   Compact flag is False, reading original format...
[0.33s]   Probing for potential extra byte before non-compact offsets...
[0.33s]   Found and consumed an unexpected 0x00 byte.
  Reading vector (dtype=<class 'numpy.uint64'>, fmt='Q')... Count=6, Bytes=48
[0.33s]   Read offsets (6)
[0.41s]   Attempting to read neighbors vector...
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')...

INFO:leann_backend_hnsw.hnsw_backend:✅ CSR conversion successful.
INFO:leann_backend_hnsw.hnsw_backend:INFO: Replaced original index with CSR-pruned version at 'index.index'


## Search with real-time embeddings

In [ ]:
from leann.api import LeannSearcher

searcher = LeannSearcher(INDEX_PATH)
results = searcher.search("programming languages", top_k=2)
results

INFO:leann.api:🔍 LeannSearcher.search() called:
INFO:leann.api:  Query: 'programming languages'
INFO:leann.api:  Top_k: 2
INFO:leann.api:  Additional kwargs: {}
INFO:leann.embedding_server_manager:Port 5557 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Port 5558 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Port 5559 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Port 5560 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Port 5561 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Port 5562 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Starting embedding server on port 5563...
INFO:leann.embedding_server_manager:Command: /Users/yichuan/Desktop/code/test_leann_pip/LEANN/.venv/bin/python -m leann_backend_hnsw.hnsw_embedding_server --zmq-port 5563 --model-name facebook/contriever --passages-

[read_HNSW - CSR NL v4] Reading metadata & CSR indices (manual offset)...
[read_HNSW NL v4] Read levels vector, size: 5
[read_HNSW NL v4] Reading Compact Storage format indices...
[read_HNSW NL v4] Read compact_level_ptr, size: 10
[read_HNSW NL v4] Read compact_node_offsets, size: 6
[read_HNSW NL v4] Read entry_point: 4, max_level: 0
[read_HNSW NL v4] Read storage fourcc: 0x6c6c756e
[read_HNSW NL v4 FIX] Detected FileIOReader. Neighbors size field offset: 326
[read_HNSW NL v4] Reading neighbors data into memory.
[read_HNSW NL v4] Read neighbors data, size: 20
[read_HNSW NL v4] Finished reading metadata and CSR indices.
INFO: Skipping external storage loading, since is_recompute is true.
INFO: Registering backend 'hnsw'


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/yichuan/Desktop/code/test_leann_pip/LEANN/.venv/lib/python3.11/site-packages/leann_backend_hnsw/hnsw_embedding_server.py", line 323, in <module>
    create_hnsw_embedding_server(
  File "/Users/yichuan/Desktop/code/test_leann_pip/LEANN/.venv/lib/python3.11/site-packages/leann_backend_hnsw/hnsw_embedding_server.py", line 98, in create_hnsw_embedding_server
    passages = PassageManager(passage_sources)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yichuan/Desktop/code/test_leann_pip/LEANN/.venv/lib/python3.11/site-packages/leann/api.py", line 127, in __init__
    raise FileNotFoundError(f"Passage index file not found: {index_file}")
FileNotFoundError: Passage index file not found: /Users/yichuan/Desktop/code/test_leann_pip/LEANN/index.passages.idx
ERROR:leann.embedding_server_manager:Server terminated during startup.


RuntimeError: Failed to start embedding server on port 5563

## Chat with LEANN using retrieved results

In [ ]:
from leann.api import LeannChat

llm_config = {
    "type": "hf",
    "model": "Qwen/Qwen3-0.6B",
}

chat = LeannChat(index_path=INDEX_PATH, llm_config=llm_config)
response = chat.ask(
    "Compare the two retrieved programming languages and tell me their advantages.",
    top_k=2,
    llm_kwargs={"max_tokens": 128},
)
response